CSC480 Assignment 1 tutorial
Damien Trunkey, Emily Lucas, Sophia Parrett, Fernando Valdivia, Divya Satrawada, Rey Ortiz
Social Justice League!

# Tutorial: Preprocessing Text Data Into TF-IDF Vectors

## Introduction:
    
This is a tutorial on how to preprocess data to make it ready to train a model. 

You can get data many ways. You can use [webscraping](https://realpython.com/python-web-scraping-practical-introduction/#reader-comments), you can get a dataset from [kaggle datasets](https://www.kaggle.com/datasets), or you can get data using an api that has access to  lots of data. We chose to use the reddit.com api to get data from the reddit.com forums. 
    


## Imports

We are using pandas and numpy, which are two data science libraries in python. The pandas import alows us to create data frames (matrices) and numpy lets us do many mathematical functions.

Numpy was originally created to add array functionality to Python. Numpy currently has many functions to support arrays and mathematical functions commonly used by scientists. We chose Numpy because we were going to be working with matrices and mathematical functions. We specifically use Numpy in this example at the end to take the log of all the values in a DataFrame (matrix).

In [ ]:
# Imports
import pandas as pd
import numpy as np
from collections import Counter
import sys
import time

In [ ]:
# Get the data into a dataframe to manupulate and do preprocessing
dataFile = 'AmItheAsshole_subreddit.csv'
df = pd.read_csv(dataFile)
df = df.head(7)
df = df[['selftext', 'ups']]
df = df[2:7]
ground_truth = df['ups']
print(ground_truth)
display(df)

## Normalization

In data science, normalization is the process of simplifying text in order to make it easier to work with. This is achieved by multiple steps. To begin with, change all of the words with the lower-case version of themselves using .lower(). This is needed because words such as "They" and "they" would be counted as two different words. However, these should be counted as the same word, so .lower() is used to normalize the capitalization of the text. 

After all the words are changed to lower-case, the next step is to replace all non-alphanumeric characters with whitespace. This is achieved by utilizing the .replace("[^\w\s]", " ") method. This would eliminate all punctuation, as well as any additional spaces, tabs, or newlines between words and replace it with whitespace. [^\w\s] is a regular expression that is used to capture all non-alphanumeric words. Once again, this is needed because punctuation is irrelevant when it comes to natural language processing and elimination it would further simplify the text. 

Additionally, replacing everything with whitespaces is needed for the final step, which is to split the text on whitespace. This is achieved by utilizing .split(). This method takes the string of text and turns it into a list of separate words. It does this by going through each letter of the string and anytime a whitespace is found, the previous grouping of letters is taken and inserted into a list as one element. To summarize, the steps of normalization are taking the string and applying .lower(), .replace("[^\w\s]", " "), and finally utilizing .split(). 

In [ ]:
bag_of_words = (
    df['selftext'].
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ", regex=True).  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
)

In [ ]:
#Getting raw frequency to turn into tf-idf vectors
raw_frequency = bag_of_words.apply(Counter)

df['selftext'] = raw_frequency

tf = pd.DataFrame(list(raw_frequency),index=raw_frequency.index)
columns = list(tf.columns)
tf = tf.fillna(0)
display(tf)
display(df)

## Remove Stop Words 
Next in this tutorial we will remove stop words. Removing stop words is important because it gets rid of trivial words (like "a", "the" etc.), and focuses on more important information. Words that are kept are more topical, and have a stronger connotation. This can be done by creating a list of stop words (from 'stopwords-short.txt') and removing columns in our dataframe that contain stop words. We have to do this step after normalization because otherwise we would not remove stop words that had different capitalizations from the words in our dataset. 

For example: it would remove "in" but not "In" if we did not normalize the data first.

In [ ]:
stopFile = 'stopwords.txt'
f = open(stopFile, "r")
stop_words = []
for line in f:
    words = line.split(',')
for word in words:
    word = word.replace('"', "").strip(" ").lower()
    stop_words.append(word)
    
for col in columns:
    if col in stop_words:
        tf = tf.drop([col], axis=1)
display(tf)

## TF-IDF Vectors

TF-IDF stands for term frequency-inverse document frequency and it is a measure to quantify  how important certain terms are throughout a list of documents (corpus). 
Term frequency (TF) is the measure of how often a word appears in a single document and inverse document frequency (IDF) is the measure of how common or uncommon a term is throughout the corpus. If a word appears in only two documents, then it may be considered rare, therefore it carries more importance. The equation to find IDF follows the form where t is the term and D is the corpus and d is the current document and N is the number of documents:
idf(t,D) = log(N / count(d D; t  D))
IDF is important because it takes common words in the English language and weights them less, giving less common words more impact.
When we put TF and IDF together we can show that a term is inversely related to its frequency across documents. By multiplying these together we can get the final TF-IDF value. Higher values means a term holds more importance and closer to 0 means it's less relevant.
ifidf(t,d,D) = tf(t,d) * idf(t,D)

In [ ]:
# Get document frequencies 
# (How many documents does each word appear in?)
df = (tf > 0).sum(axis=0)

# Get IDFs
idf = np.log(len(tf) / df)
idf.sort_values()

# Calculate TF-IDFs
tf_idf = tf * idf
tf_idf

# Conclusion

Now you have preprocessed data to train your classification model!